# PyCoMo Loopless FVA #
This tutorial show-cases the use of loopless FVA.

The expected runtime for this notebook is less than 10 minutes.

In [1]:
from pathlib import Path
import sys
import os
import cobra
import matplotlib.pyplot as plt
from cobra import Reaction, Metabolite, Model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
import pandas as pd
import numpy as np
import math 
import time
import warnings

In [2]:
path_root = "../src"  # Change path according to your PyCoMo location
sys.path.append(str(path_root))
import pycomo

2024-07-08 16:35:38,824 - pycomo.helper.multiprocess - INFO - Multiprocess Logger initialized.
2024-07-08 16:35:38,824 - pycomo.pycomo_models - INFO - Logger initialized.


## Community toy model ##
This simple community toy model consists of two identical community members. Each member has a two step conversion of the starting metabolite A. The products of these conversions (B and C) can be secreted and taken up from the medium, thus allowing exchange and forming a cycle. The members also have a biomass reaction with substrates C and D (an additional substrate).

In [3]:
model_single = Model()
model_single.add_metabolites([Metabolite(i) for i in "ABCD"])

for met in model_single.metabolites:
    met.compartment = "c"

model_single.add_metabolites([Metabolite(i+"_e") for i in "ABCD"])

for i in "ABCD":
    model_single.metabolites.get_by_id(i+"_e").compartment = "e"

model_single.add_reactions([Reaction(i) for i in ["EX_A", "EX_B", "EX_C", "EX_D", "TP_A", "TP_B", "TP_C", "TP_D", "bio", "v1", "v2"]])

model_single.reactions.EX_A.add_metabolites({"A_e": 1})
model_single.reactions.EX_B.add_metabolites({"B_e": 1})
model_single.reactions.EX_C.add_metabolites({"C_e": 1})
model_single.reactions.EX_D.add_metabolites({"D_e": 1})
model_single.reactions.TP_A.add_metabolites({"A_e": -1, "A": 1})
model_single.reactions.TP_B.add_metabolites({"B_e": -1, "B": 1})
model_single.reactions.TP_C.add_metabolites({"C_e": -1, "C": 1})
model_single.reactions.TP_D.add_metabolites({"D_e": -1, "D": 1})
model_single.reactions.bio.add_metabolites({"C": -1, "D": -1})

model_single.reactions.v1.add_metabolites({"A": -1, "B": 1})
model_single.reactions.v2.add_metabolites({"B": -1, "C": 1})

model_single.reactions.TP_B.lower_bound = -500
model_single.reactions.TP_C.lower_bound = -500
model_single.reactions.v2.lower_bound = -1000

model_single.objective = 'bio'

### Constructing the community model ###

In [4]:
single_org_models_toy = []
for name in ["model_a", "model_b"]:
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model_single, name)
    single_org_models_toy.append(single_org_model)

model_a
model_b


In [5]:
community_name = "toy_com"
com_model_obj_toy = pycomo.CommunityModel(single_org_models_toy, community_name)
com_model_obj_toy.convert_to_fixed_abundance()

No community model generated yet. Generating now:
Note: no products in the objective function, adding biomass to it.


Ignoring reaction 'EX_A_medium' since it already exists.
Ignoring reaction 'EX_B_medium' since it already exists.
Ignoring reaction 'EX_C_medium' since it already exists.
Ignoring reaction 'EX_D_medium' since it already exists.


Note: no products in the objective function, adding biomass to it.
Generated community model.


In [6]:
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()

Name,toy_com
Memory address,0x01bd0803f070
Number of metabolites,64
Number of reactions,71
Number of groups,0
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"model_a_c, model_a_e, model_a_medium, medium, fraction_reaction, model_b_e, model_b_c, model_b_medium"


In [7]:
com_model_obj_toy.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
A_medium,EX_A_medium,1000,0,0.00%
D_medium,EX_D_medium,1000,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd11416_medium,community_biomass,-1000,0,0.00%


### Loops in the toy model ###

In [8]:
com_model_obj_toy.get_loops()

,reaction,min_flux,max_flux
0,model_a_TP_B_model_a_e,-500.0,500.0
1,model_a_TP_C_model_a_e,-500.0,500.0
2,model_a_TP_C,-500.0,500.0
3,model_a_TP_B,-500.0,500.0
4,model_a_v2_model_a_c,-500.0,500.0
5,model_b_TP_B_model_b_e,-500.0,500.0
6,model_b_TP_C_model_b_e,-500.0,500.0
7,model_b_TP_C,-500.0,500.0
8,model_b_TP_B,-500.0,500.0
9,model_b_v2_model_b_c,-500.0,500.0


## FVA examples ##

### FVA with loops ###
Normal fva will include the loops including metabolites B and C, reaction v2 and the transporters of B and C in the solutions.

The following scenarios and correct outcomes will be tested on a :
 - No medium: no reaction can carry flux
 - 100% objective value: reaction v2 carries maximum flux (500; = 1000 * 0.5 as normalized to equal abundance) and transport reactions of B and C are inactive
 - 0% objective value: reactions v2 and transporters of B and C carry maximum flux (250, reverse also for transporters)
 - 80% objective value: reaction v2 carries flux between 300 and 500. Transporter of B can be active between -200 and 200, transporter of C between -100 and 100. This solution arises by one member giving 200 of B to the other (keeping 300), which converts B to C with maximum flux of 500, then gives back 100 of C while keeping the required 400. This results in a biomass reaction of 400 for both (80% of 500). 

#### COBRApy fva ####
The loopless version of COBRApy cannot be used on PyCoMo community models, due to their bound-free reaction structure. The resulting solutions include the loop:

In [9]:
%%time
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = cobra.flux_analysis.flux_variability_analysis(com_model_obj_toy.model, 
                                                        com_model_obj_toy.model.reactions, 
                                                        fraction_of_optimum=0., 
                                                        loopless=True)
fva

CPU times: total: 125 ms
Wall time: 9.38 s


,minimum,maximum
model_a_TP_A_model_a_e,0.0,0.0
model_a_TP_B_model_a_e,-250.0,250.0
model_a_TP_C_model_a_e,-250.0,250.0
model_a_TP_D_model_a_e,0.0,0.0
model_a_TP_A,0.0,0.0
...,...,...
SK_model_b_v2_model_b_c_ub,2.5,7.5
SK_model_b_to_community_biomass_ub,5.0,5.0
f_final,1.0,1.0
abundance_reaction,0.0,0.0


#### PyCoMo fva with loops ####
The PyCoMo wrapper for fva will also include loops, when loopless mode is not activated

In [10]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=1., loopless=False)
fva

CPU times: total: 46.9 ms
Wall time: 8.34 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,0.0,0.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,-250.0,250.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,-250.0,250.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,0.0,0.0
EX_A_medium,EX_A_medium,0.0,0.0
EX_B_medium,EX_B_medium,0.0,0.0
EX_C_medium,EX_C_medium,0.0,0.0
EX_D_medium,EX_D_medium,0.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,0.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,0.0,0.0


### PyCoMo loopless fva ###
The following examples show that the loopless fva implemented in PyCoMo leads to the correct solutions in the 4 test cases (no medium, 100% objective value, 0% objective value, 80% objective value)

__Note:__ For larger (genome-scale, >4 members) it will be beneficial to use parallel processing for loopless FVA. The number of processes can be specified either using the cobrapy configuration object, or directly in the functions of PyCoMo, with the processes argument.

In [11]:
%%time
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0., loopless=True)
fva

2024-07-08 15:15:50,863 - pycomo.helper.multiprocess - INFO - Processed 66.67% of fva steps


CPU times: total: 141 ms
Wall time: 8.57 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,0.0,0.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,0.0,0.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,0.0,0.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,0.0,0.0
EX_A_medium,EX_A_medium,0.0,-0.0
EX_B_medium,EX_B_medium,0.0,-0.0
EX_C_medium,EX_C_medium,0.0,-0.0
EX_D_medium,EX_D_medium,-0.0,-0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,0.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,0.0,0.0


With loopless FVA, PyCoMo correctly calculates 0 flux for all reactions, when no medium is present.

In [12]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=1., loopless=True)
fva

2024-07-08 15:15:59,343 - pycomo.helper.multiprocess - INFO - Processed 66.67% of fva steps


CPU times: total: 156 ms
Wall time: 8.47 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,500.0,500.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,0.0,0.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,0.0,0.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,500.0,500.0
EX_A_medium,EX_A_medium,-1000.0,-1000.0
EX_B_medium,EX_B_medium,0.0,-0.0
EX_C_medium,EX_C_medium,0.0,-0.0
EX_D_medium,EX_D_medium,-1000.0,-1000.0
model_a_to_community_biomass,model_a_to_community_biomass,500.0,500.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,500.0,500.0


At maximum growht-rate, the reactions that are part of loops are not used, as this would lead to a decrease in growth-rate.

In [13]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0., loopless=True)
fva

2024-07-08 15:16:07,787 - pycomo.helper.multiprocess - INFO - Processed 66.67% of fva steps


CPU times: total: 125 ms
Wall time: 8.4 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,0.0,500.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,-250.0,250.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,-250.0,250.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,0.0,500.0
EX_A_medium,EX_A_medium,-1000.0,0.0
EX_B_medium,EX_B_medium,0.0,500.0
EX_C_medium,EX_C_medium,0.0,500.0
EX_D_medium,EX_D_medium,-1000.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,500.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,0.0,500.0


However, there are valid flux configurations where the "loopy" reactions can carry flux, with both directions being viable without the use of loops.

In [14]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0.8, loopless=True)
fva

2024-07-08 15:16:16,407 - pycomo.helper.multiprocess - INFO - Processed 66.67% of fva steps


CPU times: total: 188 ms
Wall time: 8.6 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,300.0,500.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,-200.0,200.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,-100.0,100.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,400.0,500.0
EX_A_medium,EX_A_medium,-1000.0,-800.0
EX_B_medium,EX_B_medium,0.0,200.0
EX_C_medium,EX_C_medium,0.0,200.0
EX_D_medium,EX_D_medium,-1000.0,-800.0
model_a_to_community_biomass,model_a_to_community_biomass,400.0,500.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,300.0,500.0


The example of 80% of maximum growth-rate shows, that the loopless solution scales down the flux of the "loopy" reactions, to the level where they can be active without decreasing the maximum growth-rate below the 80% threshold.

In [15]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, composition_agnostic=True, fraction_of_optimum=0., loopless=True)
fva

fixing mu_c
(0.0, 1000.0)
{<Metabolite cpd11416_medium at 0x1636c986f70>: -1}
{<Metabolite model_a_cpd11416_model_a_medium at 0x1636c921730>: -1, <Metabolite model_a_to_community_biomass_ub at 0x1636c958070>: -0.01, <Metabolite cpd11416_medium at 0x1636c986f70>: 1}
{<Metabolite model_b_to_community_biomass_ub at 0x1636c97e7c0>: -0.01, <Metabolite model_b_cpd11416_model_b_medium at 0x1636c8b8ca0>: -1, <Metabolite cpd11416_medium at 0x1636c986f70>: 1}


2024-07-08 15:16:25,482 - pycomo.helper.multiprocess - INFO - Processed 66.67% of fva steps


CPU times: total: 172 ms
Wall time: 9.06 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,0.0,1000.0
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,-500.0,250.0
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,-500.0,0.0
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,0.0,0.0
EX_A_medium,EX_A_medium,-1000.0,0.0
EX_B_medium,EX_B_medium,0.0,500.0
EX_C_medium,EX_C_medium,0.0,500.0
EX_D_medium,EX_D_medium,0.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,0.0
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,0.0,1000.0


In [8]:
%%time
com_model_obj_toy.convert_to_fixed_growth_rate(1.0)
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()

with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, composition_agnostic=True, fraction_of_optimum=0., loopless=True, processes=1)
fva

has fixed mu_c
(0.0, 1000.0)
{<Metabolite cpd11416_medium at 0x1bd0814dd30>: -1}
(0, 1000.0)
{<Metabolite model_a_cpd11416_model_a_medium at 0x1bd08156dc0>: -1, <Metabolite model_a_to_community_biomass_ub at 0x1bd081a66d0>: -0.01, <Metabolite cpd11416_medium at 0x1bd0814dd30>: 1}
{<Metabolite f_final_met at 0x1bd0817de20>: 1, <Metabolite model_a_TP_A_model_a_e_lb at 0x1bd081a6130>: 10.0, <Metabolite model_a_TP_A_model_a_e_ub at 0x1bd081a64c0>: 10.0, <Metabolite model_a_TP_B_model_a_e_lb at 0x1bd081a60a0>: 10.0, <Metabolite model_a_TP_B_model_a_e_ub at 0x1bd081a6490>: 10.0, <Metabolite model_a_TP_C_model_a_e_lb at 0x1bd081a6400>: 10.0, <Metabolite model_a_TP_C_model_a_e_ub at 0x1bd081a6340>: 10.0, <Metabolite model_a_TP_D_model_a_e_lb at 0x1bd081a63d0>: 10.0, <Metabolite model_a_TP_D_model_a_e_ub at 0x1bd081a63a0>: 10.0, <Metabolite model_a_TP_A_ub at 0x1bd081a6310>: 10.0, <Metabolite model_a_TP_B_lb at 0x1bd081a62b0>: 5.0, <Metabolite model_a_TP_B_ub at 0x1bd081a6280>: 10.0, <Metabolit

2024-07-08 16:35:49,618 - pycomo.helper.multiprocess - DEBUG - Summary before context: 
Objective
1.0 model_a_to_community_biomass = 1000.0

Uptake
------
Metabolite    Reaction Flux  C-Number C-Flux
  A_medium EX_A_medium 1000         0  0.00%
  D_medium EX_D_medium 1000         0  0.00%

Secretion
---------
     Metabolite          Reaction  Flux  C-Number C-Flux
cpd11416_medium community_biomass -1000         0  0.00%

2024-07-08 16:35:49,635 - pycomo.helper.multiprocess - DEBUG - Summary at start: 
Objective
1.0 model_a_to_community_biomass = 1000.0

Uptake
------
Metabolite    Reaction Flux  C-Number C-Flux
  A_medium EX_A_medium 1000         0  0.00%
  D_medium EX_D_medium 1000         0  0.00%

Secretion
---------
     Metabolite          Reaction  Flux  C-Number C-Flux
cpd11416_medium community_biomass -1000         0  0.00%



                 reaction  min_flux  max_flux
0  model_a_TP_B_model_a_e    -500.0     500.0
1  model_a_TP_C_model_a_e    -500.0     500.0
2            model_a_TP_B    -500.0     500.0
3            model_a_TP_C    -500.0     500.0
4    model_a_v2_model_a_c    -500.0     500.0
5  model_b_TP_B_model_b_e    -500.0     500.0
6  model_b_TP_C_model_b_e    -500.0     500.0
7            model_b_TP_B    -500.0     500.0
8            model_b_TP_C    -500.0     500.0
9    model_b_v2_model_b_c    -500.0     500.0
Summary after get loops in run fva: 
Objective
1.0 model_a_to_community_biomass = 1000.0

Uptake
------
Metabolite    Reaction Flux  C-Number C-Flux
  A_medium EX_A_medium 1000         0  0.00%
  D_medium EX_D_medium 1000         0  0.00%

Secretion
---------
     Metabolite          Reaction  Flux  C-Number C-Flux
cpd11416_medium community_biomass -1000         0  0.00%



2024-07-08 16:35:53,865 - pycomo.helper.multiprocess - DEBUG - Summary after get loops: 
Objective
1.0 model_a_to_community_biomass = 1000.0

Uptake
------
Metabolite    Reaction Flux  C-Number C-Flux
  A_medium EX_A_medium 1000         0  0.00%
  D_medium EX_D_medium 1000         0  0.00%

Secretion
---------
     Metabolite          Reaction  Flux  C-Number C-Flux
cpd11416_medium community_biomass -1000         0  0.00%

2024-07-08 16:35:53,884 - pycomo.helper.multiprocess - DEBUG - Summary after setting objective: 
Objective
1.0 model_a_to_community_biomass = 1000.0

Uptake
------
Metabolite    Reaction Flux  C-Number C-Flux
  A_medium EX_A_medium 1000         0  0.00%
  D_medium EX_D_medium 1000         0  0.00%

Secretion
---------
     Metabolite          Reaction  Flux  C-Number C-Flux
cpd11416_medium community_biomass -1000         0  0.00%

2024-07-08 16:35:53,885 - pycomo.helper.multiprocess - DEBUG - Running with 1 processes
2024-07-08 16:35:53,886 - pycomo.helper.multiproce

2024-07-08 16:35:53,979 - pycomo.helper.multiprocess - DEBUG - Starting add loopless constraints
2024-07-08 16:35:53,980 - pycomo.helper.multiprocess - DEBUG - Add exchange constraints
2024-07-08 16:35:53,981 - pycomo.helper.multiprocess - DEBUG - Add ko_candidate constraints
2024-07-08 16:35:53,982 - pycomo.helper.multiprocess - DEBUG - Add remaining constraints
2024-07-08 16:35:53,984 - pycomo.helper.multiprocess - DEBUG - Set coefficients
2024-07-08 16:35:53,984 - pycomo.helper.multiprocess - DEBUG - add loopless constraints finished
2024-07-08 16:35:53,985 - pycomo.helper.multiprocess - DEBUG - model_b_TP_B_model_b_e Optimize for loopless flux
2024-07-08 16:35:53,987 - pycomo.helper.multiprocess - DEBUG - model_b_TP_B_model_b_e Optimization for loopless flux finished
2024-07-08 16:35:53,990 - pycomo.helper.multiprocess - DEBUG - model_b_TP_B_model_b_e solution min flux -500.0
2024-07-08 16:35:53,992 - pycomo.helper.multiprocess - DEBUG - model_b_TP_B_model_b_e Starting loop correct

CPU times: total: 469 ms
Wall time: 8.75 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_e,model_a_TP_A_model_a_e,0.0,1000.000000
model_a_TP_B_model_a_e,model_a_TP_B_model_a_e,-500.0,333.333333
model_a_TP_C_model_a_e,model_a_TP_C_model_a_e,-500.0,166.666667
model_a_TP_D_model_a_e,model_a_TP_D_model_a_e,0.0,1000.000000
EX_A_medium,EX_A_medium,-1000.0,0.000000
EX_B_medium,EX_B_medium,0.0,500.000000
EX_C_medium,EX_C_medium,0.0,500.000000
EX_D_medium,EX_D_medium,-1000.0,0.000000
model_a_to_community_biomass,model_a_to_community_biomass,0.0,1000.000000
model_b_TP_A_model_b_e,model_b_TP_A_model_b_e,0.0,1000.000000
